In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie": "What do you know about Inception?",
        "answer": """
        Here is what I know:
        Director: Christopher Nolan
        Main Cast: Leonardo DiCaprio as Dom Cobb, Joseph Gordon-Levitt as Arthur
        Budget: $160 million
        Box Office Revenue: $836.8 million
        Genre: Science Fiction, Action, Thriller
        Synopsis: Dom Cobb is a skilled thief who specializes in infiltrating the subconscious minds of his targets to steal valuable secrets. 
            He is offered a chance to have his past crimes erased if he can successfully perform an "inception"—planting an idea in someone's mind without them realizing it. 
            As he assembles a team and dives deeper into the dream world, the lines between reality and dreams blur, leading to a thrilling and mind-bending experience.
        """,
    },
    {
        "movie": "What do you know about The Shawshank Redemption?",
        "answer": """
        Here is what I know:
        Director: Frank Darabont
        Main Cast: Tim Robbins as Andy Dufresne, Morgan Freeman as Ellis "Red" Redding
        Budget: $25 million
        Box Office Revenue: $73.3 million
        Genre: Drama, Crime
        Synopsis: Andy Dufresne, a banker, is wrongfully convicted of murder and sentenced to life in Shawshank State Penitentiary.
            Over the years, he befriends fellow inmate Red and gains respect and influence within the prison, ultimately orchestrating a daring escape. 
            The film explores themes of hope, friendship, and perseverance in the face of despair.
        """,
    },
    {
        "movie": "What do you know about Mad Max: Fury Road?",
        "answer": """
        Here is what I know:
        Director: George Miller
        Main Cast: Tom Hardy as Max Rockatansky, Charlize Theron as Imperator Furiosa
        Budget: $150-185 million
        Box Office Revenue: $374.7 million
        Genre: Action, Adventure, Science Fiction
        Synopsis: In a post-apocalyptic wasteland, Max Rockatansky teams up with Imperator Furiosa as they flee from a tyrannical warlord, Immortan Joe, and his army. 
            With intense car chases, explosive action, and stunning visuals, the film follows their desperate attempt to escape and find redemption in a world devoid of hope.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert that knows all informations about movies. You give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "The Grand Budapest Hotel"})

chain.invoke({"movie": "Eternal Sunshine of the Spotless Mind"})



        Here is what I know:
        Director: Wes Anderson
        Main Cast: Ralph Fiennes as Monsieur Gustave H., Tony Revolori as Zero Moustafa
        Budget: $25 million
        Box Office Revenue: $174.8 million
        Genre: Comedy, Drama
        Synopsis: The film follows the adventures of Monsieur Gustave H., the legendary concierge at the Grand Budapest Hotel, and his loyal lobby boy, Zero Moustafa. 
            Set against the backdrop of a fictional European alpine country on the brink of war, the story involves a priceless painting, a murder mystery, and a whimsical tale of friendship and loyalty.
        Here is what I know:
        Director: Michel Gondry
        Main Cast: Jim Carrey as Joel Barish, Kate Winslet as Clementine Kruczynski
        Budget: $20 million
        Box Office Revenue: $74.1 million
        Genre: Drama, Romance, Science Fiction
        Synopsis: Joel and Clementine undergo a procedure to erase memories of their tumultuous relationship from the

AIMessageChunk(content='\n        Here is what I know:\n        Director: Michel Gondry\n        Main Cast: Jim Carrey as Joel Barish, Kate Winslet as Clementine Kruczynski\n        Budget: $20 million\n        Box Office Revenue: $74.1 million\n        Genre: Drama, Romance, Science Fiction\n        Synopsis: Joel and Clementine undergo a procedure to erase memories of their tumultuous relationship from their minds. \n            As Joel relives his memories during the erasure process, he realizes the depth of his love for Clementine and fights to hold onto those precious moments, leading to a poignant exploration of love, loss, and memory.')